<a href="https://colab.research.google.com/github/TivoGatto/VAEs/blob/master/DRAW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# LIBRARIES
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, Lambda, LSTM, Activation
import keras.backend as K

Using TensorFlow backend.


In [0]:
# PARAMETERS
input_dim = 784
latent_dim = 2

T = 10
N = 5

epochs = 10
batch_size = 200
epsilon_std = 1.0

In [0]:
# FUNCTIONS

def draw_loss(x_true, x_pred):
    xent_loss = 0.5 * K.sum(K.square(x_true - x_pred), axis=-1)
    reg_loss = K.sum(kl) / 10.

    return K.mean(xent_loss + reg_loss)

def Regularizer(x_true, x_pred):
	return K.mean(K.sum(kl))
 
def L2(x_true, x_pred):
	return K.mean(0.5 * K.sum(K.square(x_true - x_pred), axis=-1))

def Conc(args):
	return K.concatenate(args, axis=-1)

def Sampling(args):
	z_mean, z_log_var = args
	epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=epsilon_std)

	return z_mean + K.exp(0.5 * z_log_var) * epsilon 

def Read(args):
	x, x_hat = args

	return conc([x, x_hat])

def Write(args):
	h_dec = args

	return Dense(input_dim, activation='relu')(h_dec)

In [0]:
# MODEL

sigm = Activation("sigmoid")
conc = Lambda(Conc)
sampling = Lambda(Sampling)

read = Lambda(Read)
write = Lambda(Write)

encoder = LSTM(latent_dim)
decoder = LSTM(input_dim)

x = Input(shape=(input_dim, ))
h_enc_init = Input(tensor=K.zeros(shape=(batch_size, latent_dim, )))
h_dec_init = Input(tensor=K.zeros(shape=(batch_size, latent_dim, )))
C_0 = Input(tensor=K.zeros(shape=(batch_size, input_dim, )))

h_enc = Dense(latent_dim)(h_enc_init)
h_dec = Dense(input_dim)(h_dec_init)
C_t   = Dense(input_dim)(C_0)

kl = []
for t in range(T):
    x_hat = keras.layers.Subtract()([x, C_t])
    r_t = read([x, x_hat])

    info = conc([r_t, h_dec])
    info = keras.layers.Reshape((-1, K.int_shape(h_dec)[1] + K.int_shape(r_t)[1]))(info)
    h_enc = encoder(info)

    z_mean = Dense(latent_dim)(h_enc)
    z_log_var = Dense(latent_dim)(h_enc)

    z = sampling([z_mean, z_log_var])

    z_reshaped = keras.layers.Reshape((1, latent_dim))(z)
    h_dec = decoder(z_reshaped)

    C_t = keras.layers.Add()([C_t, write(h_dec)])
    kl.append(0.5 * K.sum(K.square(z_mean) + K.exp(z_log_var) - 1 - z_log_var, axis=-1))


model = Model([x, C_0, h_dec_init, h_enc_init], C_t)
model.compile(optimizer='adam', loss=draw_loss, metrics=[L2, Regularizer])

In [5]:
# IMPORT DATA
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = np.reshape(x_train, (-1, input_dim))/255
x_test = np.reshape(x_test, (-1, input_dim))/255

print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [6]:
#C_0_train = np.zeros((x_train.shape[0], input_dim))
#h_dec_train = np.zeros((x_train.shape[0], latent_dim))
#h_enc_train = np.zeros((x_train.shape[0], latent_dim))

model.fit(x_train, x_train, batch_size=batch_size, epochs=epochs)

Epoch 1/10
60000/60000 [==============================] - 29s 488us/step - loss: 29.3846 - L2: 29.3449 - Regularizer: 0.3971
Epoch 2/10
60000/60000 [==============================] - 25s 409us/step - loss: 26.4003 - L2: 26.4001 - Regularizer: 0.0014
Epoch 3/10
19600/60000 [========>.....................] - ETA: 16s - loss: 26.3335 - L2: 26.3335 - Regularizer: 5.1980e-04

KeyboardInterrupt: ignored

In [0]:
x_recon = model.predict(x_train, batch_size=batch_size)

for i in range(5):
    X_recon = x_recon[i]
    X_recon = np.reshape(X_recon, (28, 28))

    plt.imshow(X_recon, cmap='gray')
    plt.show()

In [0]:
n = 10

C_0 = Input(tensor=K.zeros(shape=(n, input_dim)))
Z_t = Input(shape=(T, latent_dim))

C_t = Dense(input_dim)(C_0)
h_dec = decoder(Z_t)
C_t = keras.layers.Add()([C_t, write(h_dec)])

generator = Model([Z_t, C_0], C_t)

z_t = np.random.normal(size=(n, T, latent_dim))
x_gen = generator.predict(z_t)

x_gen = np.reshape(x_gen, (n, 28, 28))

for i in x_gen:
    plt.imshow(i, cmap='gray')
    plt.show()